# Week 11, Part 1

### Topic
 1. Play with linear fit - in RStudio
 1. Linear regression with Beers vs. Blood Alcohol level



In [1]:
# resize
require(repr)
options(repr.plot.width=8, repr.plot.height=5)

Loading required package: repr


## 1. Play with linear fit - in RStudio

Install some packages:

```{r}
install.packages("manipulate") # run once
install.packages("tigerstats") # run once

require(tigerstats)
require(manipulate)

FindRegLine()
```

*Play with the above for a bit!*

Note: this is essentially what the computer is doing when you ask it for a linear regression - its trying numbers to get the right slope and intercept.

## 2. Linear regression with Beers vs. Blood Alcohol level

Now, lets work with some real data: the Beers vs. BAC.

BAC = in grams of alcohol per deciliter of blood. BAC is usually what is quoted for blood alcohol level.

This data is from a study of the effects of beer consumption on blood alcohol levels. A group of n=16 student volunteers at The Ohio State University drank a randomly assigned number of beers. Thirty minutes later, a police officer measured their BAC.

Your instincts should tell you about the direction of this relationship – that there is a positive relationship between Beers and BAC, i.e. - if you drink more beer, you're gonna get more drunk.